In [2]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

for the cell above, we're loading the necessary package. 
This is a follow-through to the tutorial provided on the shared google folder
In this case, I'm going to use my own dataset

In [6]:
# getting the training data
training = pd.read_json('data/global_twitter_data.json', lines = True)
# df = pd.read_json ("filename.json", lines = True)

In [7]:
training.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,2022-08-07 22:31:20+00:00,1556407673284861952,1556407673284861952,RT @i_ameztoy: Extra random image (I):\n\nLets...,False,"[0, 143]","{'hashtags': [{'text': 'City', 'indices': [132...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-07 22:31:16+00:00,1556407654373027840,1556407654373027840,RT @IndoPac_Info: #China's media explains the ...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [18...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-07 22:31:07+00:00,1556407616208773122,1556407616208773120,"China even cut off communication, they don't a...",False,"[0, 161]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.556408e+18,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-07 22:31:06+00:00,1556407614602461185,1556407614602461184,"Putin to #XiJinping : I told you my friend, Ta...",False,"[0, 179]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-07 22:31:04+00:00,1556407604238245889,1556407604238245888,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


wow. can't believe the line above worked. 

trying to get the dataframe that i painstakingly created on extract_data_frame.py orz

In [9]:
# importing the libraries needed to extract the dataframe we will be uisng

import json
from time import strftime
import pandas as pd
from textblob import TextBlob

In [10]:
# to read the json file

def read_json(json_file: str)->list:
    """
    json file reader to open and read json files into a list
    Args:
    -----
    json_file: str - path of a json file
    
    Returns
    -------
    length of the json file and a list of json
    """
    
    tweets_data = []
    for tweets in open(json_file,'r'):
        tweets_data.append(json.loads(tweets))

    return len(tweets_data), tweets_data

In [13]:
# parsing tweets loaded into a pandas dataframe

class TweetDfExtractor:
    """
    this function will parse tweets json into a pandas dataframe
    
    Return
    ------
    dataframe
    """
    def __init__(self, tweets_list):
        self.tweets_list = tweets_list

    def find_statuses_count(self)->list:
        statuses_count = []
        for tweet in self.tweets_list:
            statuses_count.append(tweet['user']['statuses_count'])

        return statuses_count
        
    def find_full_text(self)->list:
        text = []
        for tweet in self.tweets_list:
            text.append(tweet['full_text'])

        return text
    
    
    def find_sentiments(self, text)->list:
        polarity=[TextBlob(tweet).sentiment.polarity for  tweet in text] 
        subjectivity=[TextBlob(tweet).sentiment.subjectivity for tweet in text]
        
        return polarity, subjectivity

    def find_lang(self)->list:
        lang = []
        for tweet in self.tweets_list:
            lang.append((tweet['lang']))

        return lang   

    def find_created_time(self)->list:
        created_at = []
        for tweet in self.tweets_list:
            created_at.append((tweet['created_at']))

        return created_at
    
    def find_source(self)->list:
        source = []
        for i in self.tweets_list:
            source.append(i['source'])
        return source

    def find_screen_name(self)->list:
        screen_name = []
        for tweet in self.tweets_list:
            screen_name.append((tweet['user']['screen_name']))

        return screen_name

    def find_followers_count(self)->list:
        followers_count = []
        for tweet in self.tweets_list:
            followers_count.append(tweet['user']['followers_count'])

        return followers_count

    def find_friends_count(self)->list:
        friends_count = []
        for tweet in self.tweets_list:
            friends_count.append(tweet['user']['friends_count'])

        return friends_count

    def is_sensitive(self)->list:
        is_sensitive = []
        
        for tweet in self.tweets_list:
            if 'possibly_sensitive' in tweet.keys():
                is_sensitive.append(tweet['possibly_sensitive'])
            else:
                is_sensitive.append(None)   

        return is_sensitive

    def find_favourite_count(self)->list:
        favorite_count = []

        for tweet in self.tweets_list:
            favorite_count.append(tweet['favorite_count'])

        return favorite_count
    
    def find_retweet_count(self)->list:
        retweet_count = []
        
        for tweet in self.tweets_list:
            retweet_count.append(tweet['retweet_count'])

        return retweet_count    

    def find_hashtags(self)->list: 
        hashtags = []

        for tweet in self.tweets_list:
            hashtags.append(tweet['entities']['hashtags'])

        return hashtags

    def find_mentions(self)->list: 
        mentions = []

        for tweet in self.tweets_list:
            mentions.append(tweet['entities']['user_mentions'])
        return mentions


    def find_location(self)->list:
        location = []

        for tweet in self.tweets_list:
            if 'location' in tweet['user'].keys():
                location.append(tweet['user']['location'])
            else:
                location.append(None) 
        
        return location

           
    def get_tweet_df(self, save=True)->pd.DataFrame:
        
        columns = ['created_at', 'source', 'original_text','polarity','subjectivity', 'lang', 
                    'favorite_count', 'retweet_count', 'original_author', 'followers_count',
                    'friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place']
        
        created_at = self.find_created_time()
        source = self.find_source()
        text = self.find_full_text()
        polarity, subjectivity = self.find_sentiments(text)
        lang = self.find_lang()
        fav_count = self.find_favourite_count()
        retweet_count = self.find_retweet_count()
        screen_name = self.find_screen_name()
        follower_count = self.find_followers_count()
        friends_count = self.find_friends_count()
        sensitivity = self.is_sensitive()
        hashtags = self.find_hashtags()
        mentions = self.find_mentions()
        location = self.find_location()
        data = zip(created_at, source, text, polarity, subjectivity, lang, fav_count, 
                    retweet_count, screen_name, follower_count, friends_count, sensitivity, 
                    hashtags, mentions, location)
        df = pd.DataFrame(data=data, columns=columns)
        mysampledf = df.head(10)

        if save:
            df.to_csv('processed_tweet_data.csv', index=False)
            mysampledf.to_json('data/sample.json')
            print('Files Successfully Saved.!!!')
        
        return df

                
if __name__ == "__main__":
    # required column to be generated you should be creative and add more features
    columns = ['created_at', 'source', 'original_text','clean_text', 'sentiment','polarity',
                'subjectivity', 'lang', 'favorite_count', 'retweet_count', 'original_author', 
                'screen_count', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 
                'user_mentions', 'place', 'place_coord_boundaries']
    _, tweet_list = read_json("data/global_twitter_data.json")

    tweet = TweetDfExtractor(tweet_list)
    tweet_df = tweet.get_tweet_df() 
    print(tweet_df.head(7))

Files Successfully Saved.!!!
                       created_at  \
0  Sun Aug 07 22:31:20 +0000 2022   
1  Sun Aug 07 22:31:16 +0000 2022   
2  Sun Aug 07 22:31:07 +0000 2022   
3  Sun Aug 07 22:31:06 +0000 2022   
4  Sun Aug 07 22:31:04 +0000 2022   
5  Sun Aug 07 22:31:02 +0000 2022   
6  Sun Aug 07 22:30:59 +0000 2022   

                                              source  \
0  <a href="http://twitter.com/download/android" ...   
1  <a href="http://twitter.com/download/android" ...   
2  <a href="http://twitter.com/download/android" ...   
3  <a href="http://twitter.com/download/android" ...   
4  <a href="http://twitter.com/download/iphone" r...   
5  <a href="http://twitter.com/download/android" ...   
6  <a href="http://twitter.com/download/android" ...   

                                       original_text      polarity  \
0  RT @i_ameztoy: Extra random image (I):\n\nLets... -1.250000e-01   
1  RT @IndoPac_Info: #China's media explains the ... -1.000000e-01   
2  China even c

# Understanding the data

In [42]:
#data loader class

class DataLoader:
    def __init__(self,dir_name,file_name):
        self.dir_name = dir_name
        self.file_name = file_name
    
 
    def read_csv(self):
        os.chdir(self.dir_name)
        tweets_df = pd.read_csv(self.file_name)
        return tweets_df

In [43]:
#object creation

DataLoader_obj = DataLoader('TwitterDataAnalysis/..','processed_tweet_data.csv')

In [44]:
tweets_df = DataLoader_obj.read_csv()
tweets_df.dropna()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
5,Sun Aug 07 22:31:02 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @benedictrogers: We must not let this happe...,0.200000,0.500000,en,0,36,GraceCh15554845,207,54,False,"[{'text': 'Taiwan', 'indices': [84, 91]}]","[{'screen_name': 'benedictrogers', 'name': 'Be...","Melbourne, Victoria"
11,Sun Aug 07 22:30:35 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Wilson Chinonso Blog: Nigerian tribes, the lis...",0.000000,0.000000,en,1,0,wilson_chnns,28,265,False,"[{'text': 'China', 'indices': [84, 90]}, {'tex...",[],Imo State Nigeria
19,Sun Aug 07 22:30:01 +0000 2022,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",27.89US $ 17% OFF|Usb Condenser Microphone For...,0.000000,0.000000,en,0,0,doos94619918,1936,4792,False,"[{'text': 'aliexpress', 'indices': [199, 210]}...",[],"United States New York,"
36,Sun Aug 07 22:26:25 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @ChinaInfo777: #PinkFloyd Roger Waters tell...,0.000000,0.000000,en,0,5,nhohn2011,870,508,False,"[{'text': 'PinkFloyd', 'indices': [18, 28]}, {...","[{'screen_name': 'ChinaInfo777', 'name': 'Chin...","Florida, USA"
39,Sun Aug 07 22:25:37 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","RT @WilliamYang120: ""For too long, #Taiwan has...",-0.025000,0.200000,en,0,84,hoggothoaryhost,44,60,False,"[{'text': 'Taiwan', 'indices': [35, 42]}]","[{'screen_name': 'WilliamYang120', 'name': 'Wi...",Hong Kong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21981,Sat Aug 06 18:04:09 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @jenniferatntd: Head of #Taiwan's #missile ...,-0.200000,0.400000,en,0,99,threadmaxwhispe,657,864,False,"[{'text': 'Taiwan', 'indices': [27, 34]}, {'te...","[{'screen_name': 'jenniferatntd', 'name': 'Jen...",Land of Ethan South Dakota
21989,Sat Aug 06 18:03:48 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",Minister Wu is crystal clear in his @BBCNews i...,0.158333,0.419444,en,2,0,TECO_Toronto,955,202,False,"[{'text': 'Taiwan', 'indices': [168, 175]}, {'...","[{'screen_name': 'BBCNews', 'name': 'BBC News ...","Toronto, Canada"
21990,Sat Aug 06 18:03:47 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @SpokespersonCHN: #PLA Live-fire military d...,-0.100000,0.250000,en,0,405,mumaralid,1164,605,True,"[{'text': 'PLA', 'indices': [21, 25]}, {'text'...","[{'screen_name': 'SpokespersonCHN', 'name': 'H...",Driver
21992,Sat Aug 06 18:03:33 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @jenniferatntd: Head of #Taiwan's #missile ...,-0.200000,0.400000,en,0,99,9thousandbytes,401,858,False,"[{'text': 'Taiwan', 'indices': [27, 34]}, {'te...","[{'screen_name': 'jenniferatntd', 'name': 'Jen...",Northern Virginia


In [45]:
len(tweets_df)

22000

In [46]:
tweets_df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Sun Aug 07 22:31:20 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,en,0,2,i_ameztoy,20497,2621,NaN,"[{'text': 'City', 'indices': [132, 137]}]","[{'screen_name': 'i_ameztoy', 'name': 'Iban Am...",NaN
1,Sun Aug 07 22:31:16 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,en,0,201,ZIisq,65,272,NaN,"[{'text': 'China', 'indices': [18, 24]}, {'tex...","[{'screen_name': 'IndoPac_Info', 'name': 'Indo...",NaN
2,Sun Aug 07 22:31:07 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","China even cut off communication, they don't a...",0.000000e+00,0.000000,en,0,0,Fin21Free,85,392,NaN,"[{'text': 'XiJinping', 'indices': [127, 137]}]","[{'screen_name': 'ZelenskyyUa', 'name': 'Волод...",Netherlands
3,Sun Aug 07 22:31:06 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,en,0,0,Fin21Free,85,392,NaN,"[{'text': 'XiJinping', 'indices': [9, 19]}]",[],Netherlands
4,Sun Aug 07 22:31:04 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...","RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,en,0,381,VizziniDolores,910,2608,NaN,[],"[{'screen_name': 'ChinaUncensored', 'name': 'C...","Ayent, Schweiz"


In [54]:
# to prepare the data for analysis

class PrepareData:
  def __init__(self,df):
    self.df=df
    
  def preprocess_data(self):
    tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
    tweets_df['original_text'] = tweets_df['original_text'].astype(str)
    tweets_df['original_text'] = tweets_df['original_text'].apply(lambda x: x.lower())
    tweets_df['original_text'] = tweets_df['original_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    #Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets_df['original_text']]
    word_list = [sent.split() for sent in sentence_list]
    # print(word_list)

    #Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    #  we can see the word to unique integer mapping
    # print(word_to_id.token2id)
    # using bag of words(bow), we create a corpus that contains the word id and its frequency in each document.    # TFIDF

    return word_list, word_to_id, corpus

In [55]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [56]:
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(9, 2), (12, 1), (15, 1), (19, 5), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(19, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1)], [(6, 2), (19, 1), (24, 1), (37, 1), (40, 1), (57, 2), (59, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2)], [(6, 1), (15, 1), (32, 1), (37, 1), (66, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94,

In [57]:
id_words = [[(id2word[id], count) for id, count in line] for line in corpus]

In [58]:
print(id_words[:5])

[[('city', 1), ('coast', 1), ('district', 1), ('extra', 1), ('focus', 1), ('gt', 1), ('i', 1), ('iameztoy', 1), ('image', 1), ('in', 1), ('lets', 1), ('longjing', 1), ('of', 1), ('one', 1), ('random', 1), ('rt', 1), ('specific', 1), ('taichung', 1), ('ta…', 1), ('the', 1), ('very', 1), ('western', 1), ('zone', 1)], [('in', 2), ('of', 1), ('rt', 1), ('the', 5), ('area', 1), ('chinas', 1), ('drills', 1), ('each', 1), ('explains', 1), ('for', 1), ('indopacinfo', 1), ('labels', 1), ('media', 1), ('military', 1), ('pi…', 1), ('read', 1), ('reasons', 1), ('strait', 1), ('taiwan', 1)], [('the', 2), ('anwer', 1), ('ask', 1), ('but', 1), ('change', 1), ('china', 1), ('clown', 1), ('communication', 1), ('cut', 1), ('dont', 1), ('enters', 1), ('even', 1), ('from', 1), ('here', 1), ('mind', 1), ('off', 1), ('phonecalls', 1), ('putins', 1), ('stage', 1), ('they', 1), ('to', 2), ('us', 1), ('xijinping', 1), ('zelenskyyua', 1)], [('i', 2), ('the', 1), ('chinas', 1), ('taiwan', 1), ('but', 1), ('to', 

## Topic Modelling Using Latent Dirichlet Allocation (LDA)

This is based on the distributional hypothesis where similar topics make use of similar words and that statistical mixture hypothesis where documents talk about several topics for which a statistical distribution can be determined

The purpose of LDA is to map each of the tweets in our corpus to a set of topics which are made up of good deal of the words in the tweet

In [59]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [60]:
pprint(lda_model.print_topics())

[(0,
  '0.055*"taiwan" + 0.054*"the" + 0.044*"on" + 0.041*"rt" + 0.036*"of" + '
  '0.027*"a" + 0.022*"in" + 0.019*"chinas" + 0.018*"indopacinfo" + '
  '0.016*"military"'),
 (1,
  '0.048*"to" + 0.036*"rt" + 0.034*"taiwan" + 0.033*"the" + 0.020*"and" + '
  '0.018*"a" + 0.017*"of" + 0.017*"china" + 0.014*"in" + 0.013*"is"'),
 (2,
  '0.049*"the" + 0.036*"rt" + 0.033*"is" + 0.024*"china" + 0.020*"this" + '
  '0.018*"taiwan" + 0.015*"by" + 0.014*"of" + 0.013*"video" + 0.012*"ukraine"'),
 (3,
  '0.034*"rt" + 0.030*"in" + 0.025*"island" + 0.018*"they" + 0.018*"taiwan’s" '
  '+ 0.017*"chinese" + 0.015*"near" + 0.015*"amp" + 0.015*"indopacinfo" + '
  '0.015*"3"'),
 (4,
  '0.036*"rt" + 0.034*"taiwan" + 0.025*"china" + 0.024*"part" + 0.022*"hotel" '
  '+ 0.017*"how" + 0.017*"your" + 0.015*"you" + 0.014*"wwiii" + 0.013*"do"')]


In [61]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('taiwan', 0.054945014),
   ('the', 0.05369046),
   ('on', 0.04424008),
   ('rt', 0.040922556),
   ('of', 0.035711158),
   ('a', 0.026685657),
   ('in', 0.02177495),
   ('chinas', 0.019176457),
   ('indopacinfo', 0.017581513),
   ('military', 0.016493807)]),
 (1,
  [('to', 0.047742132),
   ('rt', 0.035950165),
   ('taiwan', 0.033747945),
   ('the', 0.03346296),
   ('and', 0.019780524),
   ('a', 0.017708385),
   ('of', 0.017010948),
   ('china', 0.016706385),
   ('in', 0.013741817),
   ('is', 0.01309229)]),
 (2,
  [('the', 0.048562713),
   ('rt', 0.035591852),
   ('is', 0.033343855),
   ('china', 0.023586182),
   ('this', 0.019779764),
   ('taiwan', 0.01806621),
   ('by', 0.01499605),
   ('of', 0.014007342),
   ('video', 0.013000169),
   ('ukraine', 0.011767801)]),
 (3,
  [('rt', 0.034384325),
   ('in', 0.030455114),
   ('island', 0.025355658),
   ('they', 0.018407082),
   ('taiwan’s', 0.01811254),
   ('chinese', 0.017257676),
   ('near', 0.015033988),
   ('amp', 0.014902738),
 

Each line above is a topic with individual topic terms and weights.

Topic0 concerns mostly: taiwan-china-military
Topic1 concerns mostly: taiwan-china
Topic2 concerns mostly: china-taiwan-ukraine
Topic3 concerns mostly: in-island-china-taiwan
Topic4 concerns mostly: taiwan-china-hotel

There is a lot of interlap in the topics, with most of them mentioning taiwan and/or china. 
This leads us to conclude that the taiwan-china conflict is a very hot topic.

## Model Analysis

Perplexity is also a measure of model quality and in natural language processing is often used as “perplexity per number of words”. 
It describes how well a model predicts a sample, i.e. how much it is “perplexed” by a sample from the observed data. 

**The lower the score, the better the model for the given data.**

A coherence matrix is used to test the model for accuracy. 

Topic coherence is a measure that compares different topic models based on their human-interpretability. 

The coherence score ‘C_V’ provides a numerical value to the interpretability of the topics

In [62]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -7.607515134412594

 Ldamodel Coherence Score/Accuracy on Tweets:  0.39531686417969125


Basic LDA Model Coherence Score **0.58** 
This means that the model has performed reasonably well in topic modeling.

### Anlayizing results 

Exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics

In [63]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.170947  0.062324       1        1  38.568463
0      0.100701  0.077702       2        1  29.436117
2      0.093970  0.080272       3        1  13.473059
3     -0.337276  0.104442       4        1   9.653359
4     -0.028342 -0.324740       5        1   8.869002, topic_info=       Term          Freq         Total Category  logprob  loglift
19      the  16675.000000  16675.000000  Default  30.0000  30.0000
114      on   7538.000000   7538.000000  Default  29.0000  29.0000
37   taiwan  16353.000000  16353.000000  Default  28.0000  28.0000
9        in   7080.000000   7080.000000  Default  27.0000  27.0000
127      is   5810.000000   5810.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
15       rt   1518.501249  17599.626964   Topic5  -3.3179  -0.0275
37   taiwan   1407.164638  16353.184785   Topic5  -3.3941  -0.0302
135    that    456.733332   2625.066482   Topic5  -4.5193   0.6738
127      is    384.085488   5810.832105   Topic5  -4.6926  -0.2940
114      on    289.314112   7538.340043   Topic5  -4.9759  -0.8376

[280 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
781        2  0.996834    23
298        4  0.999137     3
185        2  0.999506   4th
61         1  0.465064     a
61         2  0.534795     a
...      ...       ...   ...
82         5  0.447806   you
584        1  0.215480  your
584        5  0.783768  your
20627      5  0.992611     🎁
2530       5  0.997425     🤗

[317 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4, 5])

# 